## Homework 3

Student Name: xxx

***
### Task 1. Web Crawler

Get the faculty list of Harvard Business School. (https://www.hbs.edu/faculty/Pages/browse.aspx?faculty=Current)

The output should be the same as <code>hbs_faculty.csv</code>


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
page = requests.get("https://www.hbs.edu/faculty/Pages/browse.aspx?faculty=Current")
#page = requests.get("https://www.hbs.edu/faculty/Pages/browse.aspx?faculty=Current&unit=Finance")

soup = BeautifulSoup(page.content, 'html.parser')

faculty = soup.find_all(class_='epsilon')[2:]

names = [a.get_text() for a in faculty]
websites = ['https://www.hbs.edu' + a['href'] for a in faculty]

df = pd.DataFrame({"Name": names, "Website": websites})

df.to_csv('hbs_faculty.csv')

***
### Side mission
***
Mission 1. Get <code>full bio</code> of <mark>Rawi E. Abdelal</mark>.

In [ ]:
page = requests.get("https://www.hbs.edu/faculty/Pages/profile.aspx?facId=6628")

soup = BeautifulSoup(page.content, 'html.parser')

fullbio = soup.find("div", {"class":"fullbio"})

if fullbio:
    fullbio_text = ''
    for p in fullbio.find_all('p'):
        #print(p.text)
        fullbio_text = fullbio_text + p.text + '\n'

print(fullbio_text)

***
Mission 2. Get <code>title</code>, <code>unit</code>, and <code>phone number</code> of <mark>Rawi E. Abdelal</mark>.

In [ ]:
#import re

page = requests.get("https://www.hbs.edu/faculty/Pages/profile.aspx?facId=6628")
soup = BeautifulSoup(page.content, 'html.parser')

title = soup.find('h2')
unit = soup.find(href="/faculty/units/bgie/Pages/default.aspx")
#unit = soup.find(href=re.compile("/faculty/units/bgie/Pages/default.aspx"))
phone = soup.find(class_='mu-uc')

print(title.text)
print(unit.text)
print(phone.text)

***
Mission 3. Append <code>title</code>, <code>unit</code>, and <code>phone number</code> of each professor to <code>hbs_faculty.csv</code>

In [ ]:
df = pd.read_csv('hbs_faculty.csv')

for i in df.index[:10]:
    name = df['Name'][i]
    website = df['Website'][i]
    print('Crawling the info of %s:%s' % (i, name))
    
    page = requests.get(website)    
    soup = BeautifulSoup(page.content, 'html.parser')
    
    title = soup.find('h2')
    #unit = soup.find(href=re.compile("/faculty/units/bgie/Pages/default.aspx"))
    unit = soup.find(href=re.compile("/faculty/units/(.*?)/Pages/default.aspx"))
    phone = soup.find(class_='mu-uc')

    if title:
        df.loc[i, 'Title'] = title.text
        #df.loc[i, 'Title_with_separator'] = title.get_text(separator="; ")

    if unit:
        df.loc[i, 'Unit'] = unit.text
    
    if phone:
        df.loc[i, 'Phone'] = phone.text

df.to_csv('hbs_faculty_title.csv', index=False)

***
Mission 4. Append <code>AREAS OF INTEREST</code> of <mark>Rawi E. Abdelal</mark>.

In [ ]:
page = requests.get("https://www.hbs.edu/faculty/Pages/profile.aspx?facId=6628")
soup = BeautifulSoup(page.content, 'html.parser')

areas = soup.find_all(href=re.compile("/faculty/Pages/browse.aspx\?topic="))

area_list = []
for area in areas:
    area_list.append(area.text)

area_list_unique_sorted = sorted(set(area_list))
area_text = '; '.join(area_list_unique_sorted)

print(area_text)

***
Mission 5. Append <code>AREAS OF INTEREST</code> and <code>ADDITIONAL TOPICS</code> of each professor to <code>hbs_faculty_title.csv</code>

In [ ]:
df = pd.read_csv('hbs_faculty_title.csv')

for i in df.index[:10]:
    name = df['Name'][i]
    website = df['Website'][i]
    print('Crawling the info of %s:%s' % (i, name))
    
    page = requests.get(website)    
    soup = BeautifulSoup(page.content, 'html.parser')
    
    areas = soup.find_all(href=re.compile("/faculty/Pages/browse.aspx\?topic="))

    if areas:
        area_list = []
        for area in areas:
            area_list.append(area.text)

        area_list_unique_sorted = sorted(set(area_list))
        df.loc[i, 'Areas'] = '; '.join(area_list_unique_sorted)

df.to_csv('hbs_faculty_area.csv', index=False)